In [94]:
import re
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer

In [95]:
def preprocess_text(text):
    text=text.lower()
    words=word_tokenize(text)
    stop_word=set(stopwords.words('english'))
    words=[word for word in words if word not in stop_word]
    stemmer=PorterStemmer()
    words=[stemmer.stem(word) for word in words]

    proprocessed_text=' '.join(words)
    return proprocessed_text

In [96]:
import pandas as pd
try:
    ds=pd.read_csv('D:/My WorkSpace/Summer-Internship/05-August-2024/questions_answers.csv',encoding='ISO-8859-1')
    print(ds.head(2))
except UnicodeDecodeError as err:
    print(f"Error: {err}")

  sr_no                                           question  \
0     1  What is Python? List some popular applications...   
1     2  What are the benefits of using Python language...   

                                                 ans  \
0  Python is a widely-used general-purpose, high-...   
1  Object-Oriented Language\nHigh-Level Language\...   

                                                ans1  
0  Python is a widely-used general-purpose, high-...  
1  Object-Oriented Language\nHigh-Level Language\...  


In [97]:
ds.isnull().sum()

sr_no       1
question    3
ans         3
ans1        3
dtype: int64

In [98]:
ds.fillna("Not Mentioned",inplace=True)

In [99]:
ds['ans1']=ds['ans1'].apply(preprocess_text)
ds.head(1)

,sr_no,question,ans,ans1
0,1,What is Python? List some popular applications...,"Python is a widely-used general-purpose, high-...","python widely-us general-purpos , high-level p..."


In [100]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer, util
model=SentenceTransformer('paraphrase-MiniLM-L6-v2')
    

In [101]:
def suggest_sections(ans,ds,min_suggestions=5):
    preprocessed_ans=preprocess_text(ans)
    ans_embedding=model.encode(preprocessed_ans)
    section_embedding=model.encode(ds['ans1'].tolist())
    similarities=util.pytorch_cos_sim(ans_embedding,section_embedding)[0]
    similarity_threhold=0.2
    relevant_indices=[]
    while len(relevant_indices)<min_suggestions and similarity_threhold>0:
        relevant_indices=[i for i, sim in enumerate(similarities)if sim>similarity_threhold]
        similarity_threhold-=0.5 #st=st-0.5
        sorted_indices=sorted(relevant_indices,key=lambda i: similarities[i],reverse=True)
        suggestions=[
            {
                'index':i,
                'question':ds.iloc[i]['question'],
                'ans':ds.iloc[i]['ans'],
                'similarity_score':similarities[i].items()
            }

            for i in sorted_indices
        ]
    return suggestions


In [103]:
print("Following are the questions : ")
print(ds['question'])
n=int(input("enter your question you want to answer :"))
ans = input("Enter your answer")
suggestions=suggest_sections(ans,ds,min_suggestions=1)
suggestion=suggestions[n-1]
if suggestion:
        print(f"S_no: {suggestion['index']}")
        print(f"Question: {suggestion['question']}")
        print(f"Your answer: {ans}")
        print(f"Expected answer : {suggestion['ans']}")
        print(f"Answer: {suggestion['ans']}")
        print(f"Similarity Score: {round(suggestion['similarity_score']*100,2)}%")
        print("_________________________________________________________________________________________\n")
else:
    print("No record is found")


Following are the questions : 
0    What is Python? List some popular applications...
1    What are the benefits of using Python language...
2    What is the difference between a Mutable datat...
3    What is the difference between a Set and Dicti...
4                            What is a pass in Python?
5    Difference between for loop and while loop in ...
6                                        Not Mentioned
7                                        Not Mentioned
8                                        Not Mentioned
Name: question, dtype: object


AttributeError: 'Tensor' object has no attribute 'items'